# Funciones de un solo sentido

## Ejercicio 1

Sea $(a_1,\ldots,a_k)$ una secuencia super-creciente de números positivos (la suma de los términos que preceden a $a_i$ es menor que $a_i$, para todo $i$). Elige $n > \sum a_i$, y $u$ un entero positivo tal que $gcd(n, u) = 1$. Define $a_i^* = ua_i \bmod n$. La función mochila (_knapsack_) asociada a $(a_1^*,\ldots,a_k^*)$ es $$f:\mathbb{Z}_2^k \rightarrow \mathbb{N}, f(x_1,\ldots,x_k) = \sum_{i=1}^k x_ia_i^*$$

Implementa esta función y su inversa, tal y como se explica en _P. J. Cameron, Notes on cryptography_. La llave pública es $((a_1,\ldots,a_k), n, u)$.

In [1]:
import numpy as np
import AritmeticaModular as am
from random import randint

"""
    Generate a super-increasing sequence of length k.
    
    A super-increasing sequence has k elements where
    
                    a_i > sum(a_0,...,a_(i - 1))
                    
    An example: (1, 2, 4, 8, 16)
    
    k: length of the message's block
"""
def generate_super_increasing(k):
    
    # generate the first element of the sequence
    sequence = [0]*k
    sequence[0] = randint(1, k)
    
    for i in range(1, k):
        sequence[i] = sum(sequence[:i]) + randint(sequence[i-1], sequence[i-1]*3)
            
    return np.array(sequence, dtype = np.uint32)

generate_super_increasing(8)

array([    5,    17,    70,   177,   511,  1774,  6408, 24499], dtype=uint32)

Con la función *generate_super_increasing* generamos una secuencia súper-creciente de longitud $k$, donde $k$ es el tamaño de bloque a cifrar del mensaje original.

A continuación, tenemos que tomar un número $n$ tal que $n > \sum_{i=0}^k a_i$ y un número $u$ que sea primo relativo con $n$, es decir, que $gcd(n, u) = 1$.

In [2]:
"""
    Generate the keys for the knapsack cipher.
    
    Parameters:
        block_length: length of the message's block
    Return:
        public_key: modified sequence
        private_key: original super-increasing sequence, n and u
"""

def generate_keys(block_length):
    # generate a sequence
    sequence = generate_super_increasing(block_length)
    
    n = sum(sequence) + sequence[randint(0,block_length)]*2
    
    found_u = False
    
    while not found_u:
        
        u = randint(1, n)
        
        if am.ext_euclides(n,u)[0] == 1: 
            found_u = True
            
    public_sequence = np.array(list(map(lambda a_i: (u*a_i) % n, sequence)), dtype = np.uint32)
    
    return public_sequence, (sequence, n, u)

key_pub, key = generate_keys(8)
print("Public key:", key_pub)
print("Private key:", key[0], key[1], key[2])

Public key: [ 767 2301 8437  604 3044 6644 8101 9984]
Private key: [   1    3   11   32   82  352 1103 3462] 11970 767


Una vez generadas nuestras llaves, podemos pasar a realizar el cifrado del mensaje. Para cifrar el mensaje, al igual que hacíamos en el cifrado en flujo usando el generador de Geffe, tendremos que pasar el string a binario, y para ello, usaremos las funciones *str_to_binlist* y *bin_to_str*. Una vez realizado esto, podremos pasar a a cifrar el mensaje en bloques de tamaño $k$.

In [3]:
from CifradoFlujo import bin_to_str, str_to_binlist

"""
                            Knapsack cypher
    
    This function take a message and the public
    key, and encrypt the message performing:
            ___n
            \                 a_i * e_i
            /__ 0
            
    where a_i are the bits of the message and e_i
    the elements of the public key.
    
    Parameters:
        - message: a str containing the plain message.
        - key_pub: public key
"""
def knapsack_cypher(message, key_pub):
    # take the block size
    block_size = len(key_pub)
    # transform the message to binary list
    binary_message = str_to_binlist(message)
    # check if the message is divisible by the block size
    # if not, add zeros at the end
    while len(binary_message) % block_size != 0:
        binary_message.append(0)
    
    cypher, binary_message = [], np.array(binary_message)
     # cypher text
    for i in range(0,len(binary_message), block_size):
        cypher.append(np.sum(key_pub[binary_message[i:i + block_size] == 1]))
        
    return cypher


encrypted = knapsack_cypher("Lorem ipsum dolor sit amet", key_pub)

print("Encrypted text:", encrypted)

Encrypted text: [11989, 38511, 19443, 27366, 30410, 8437, 23766, 11342, 29427, 27970, 30410, 8437, 17382, 38511, 20426, 38511, 19443, 8437, 29427, 23766, 17986, 8437, 20722, 30410, 27366, 17986]


In [4]:
from AritmeticaModular import inverse
"""
                                        Knapsack decypher
                        
In order to decrypt a message encrypted with this algorithm,
it's necessary to calculate v as u^{-1} mod n using Euclids' 
algorithm.

Then, the message is decrypted:
        b 
"""
def knapsack_decypher(message, key):
    # unfold the private key and calculate 
    # the inverse of u mod n
    p_key, n, u = key
    v =  inverse(u, n)
    # Concatenate list in python it's very simple if you use the operator + and two list. 
    # That is the reason to make decypher_message an empty list
    decypher_message = []
    # start decypher
    for b in message:
        # b_ = vb mod n
        b_, aux = (b * v) % n, 0
        # start the greedy algorithm
        greedy_result = set()
        for i in range(1, len(p_key) + 1):
            if aux + p_key[-i] <= b_:
                aux += p_key[-i]
                greedy_result.add(p_key[-i])
                if aux == b_:
                    break
        # restore and add the message
        decypher_message += list(map(lambda x: 1 if x in greedy_result else 0, p_key))
        # return the plain text
    return bin_to_str(decypher_message)


knapsack_decypher(encrypted, key)

'Lorem ipsum dolor sit amet'

En la función de descifrado haremos uso de las propiedades de las listas en Python y el uso de _set_ para poder buscar los elementos con una mayor velocidad. Al realizar el algoritmo Greedy, iremos insertando los elementos en un _set_, ya que, una vez haya encontrado el algoritmo Greedy la solución, pasaremos a recomponer el bloque de mensaje. Para ello, usaremos una función _map_ que crea una lista de forma que si $x$ se encuentra en el resultado del Greedy, devuelve un 1 y en caso contrario devuelve un 0.

Esto nos devolverá una lista de unos y ceros que podemos concatenar entre sí usando el operador $+$ de las listas. Cuando tengamos el mensaje completo lo reconstruimos a texto plano usando la función *bin_to_str*.

## Ejercicio 2

Sea $p$ un (pseudo-)primo mayor o igual que vuestro número de identidad. Encuentra un elemento primitivo $\alpha$, de $\mathbb{Z}_p^*$ (se puede usar el libro "_Handbook of Applied Cryptography [2.132 (iv)]_"); para facilitar el criterio, es bueno escoger $p$ de forma que $\frac{p - 1}{2}$ sea también primo, y para ello usamos Miller-Rabin). Definimos $$f:\mathbb{Z}_p \rightarrow \mathbb{Z}_p, x\mapsto\alpha^x$$
Calcula el inverso de tu fecha de nacimiento con el formato AAAAMMDD.

__En lo que sigue, $p$ y $q$ son enteros primos, y $n = pq$.__

In [5]:
from os import environ
import AritmeticaModular as am
from random import randint

# search a prime p that (p-1) // 2 is prime too (probably)
def search_prime(n):
    p = n
    while not (am.miller_rabin_test(p) and
               am.miller_rabin_test((p -1)//2)):
            p +=1
    return p

# initial number
p, β = search_prime(int(environ["ID"])), int(environ["BIRTH"])
α = randint(2, p)

"""
                Search a primitive α of Z_p
    
    α will be an primitive of Z_p if and only if:
    
        * α² != 1
        * α^((p-1) / 2) != 1
"""        
def search_primitive_Zp(p):
    α, found = randint(2, p-2), False
    
    while not found:
        
        if am.big_pow(α, (p - 1) // 2, p) != 1 and α**2 != 1:
            found = True
        else:
            α = randint(2, p - 2)
            
    return α

α = search_primitive_Zp(p)
print("Primitive:", α)

Primitive: 16936272


Para calcular el inverso del mi fecha de nacimiento módulo el primer primo que surge de mi DNI, tenemos que calcular primero un elemento primitivo del cuerpo $\mathbb{Z}_p$. Para ello, como hemos elegido $p$ de la forma que $p$ es primo y $\frac{p - 1}{2}$ también lo es, podemos definir $p - 1$ como: $$p - 1 = \underbrace{2}_{q_1} \cdot \underbrace{\frac{p - 1}{2}}_{q_2}$$ Entonces, $\alpha$ será un elemento primitivo si y sólo si cumple las siguientes condiciones:
* $\alpha^2\neq 1 \longrightarrow \alpha\neq1$.
* $\alpha^{\frac{p -1}{2}} \neq 1$.

Por lo tanto, buscaremos un $\alpha$ de manera aleatoria que cumpla estas condiciones. Aunque sea aleatorio, esto tardará bastante poco porque la probabilidad de encontrarse un elemento de este tipo es bastante alta, siendo de $\frac{1}{3}$.

Una vez que tenemos esto, vamos a calcular el logaritmo de la solución para obtener la inversa de la imagen de la función.

In [6]:
inv = am.baby_step_giant_step(α, β, p)
print("Inverso:", inv)

Inverso: 15186901


Ahora para comprobar si la solución es correcta, aplicamos la exponencial para comprobar si obtenemos el mismo resultado, donde $\beta$ es la fecha de nacimiento. 

In [7]:
am.big_pow(α, inv, p) == β

True

## Ejercicio 3

Sea $f:\mathbb{Z}_n \rightarrow \mathbb{Z}_n$ la función de Rabin: $f(x) = x^2$. Sea $n = 48478872564493742276963$. Sabemos que $f(12) = 144= f(37659670402359614687722)$. Usando esta información, calcula $p$ y $q$ (mirar la demostración de "_Lecture Notes on Cryptography_", Lemma 2.43.

In [8]:
x, y = 37659670402359614687722, 12
n = 48478872564493742276963

p = am.ext_euclides((x - y), n)[0]
q = n // p

print("p:", p, "q:", q, "correctos:", (p*q)==n)

p: 159497098847 q: 303948303229 correctos: True


Para resolver este ejercicio, he seguido el razonamiento del Lemma 2.43 del libro "_Lecture Notes on Criptography_", donde si conocemos un número y su cuadrado, es muy fácil extraer un divisor no trivial de $n$, teniendo en cuenta que:

1. Escogemos aleatoriamente un $x \in \mathbb{Z}_n^*$.
2. Tomamos un $y$ que sea $y = x^2 \bmod n$.
3. En caso de que $x \equiv y \bmod n $ no sea cierto, el $\gcd(x - y, n)$ es un divisor no trivial de $n$. Por lo tanto, ya habremos factorizado $n$.

Como en este caso ya disponemos de dos $x$ y sus exponenciación, podemos calcular fácilmente $p$ y $q$.

## Ejercicio 4


Elige $a_0$ y $a_1$ dos cuadrados arbitrarios módulo $n$ ($n$ como en el Ejercicio 3). Sea $$h:\mathbb{Z}_2 \times (\mathbb{Z}_n)^*, h(b,x) = x^2a_0^ba_1^{1 - b}$$ Usa la función de Merkle-Damgard para implentar una función resumen tomando $h$ como función de compresión (esta $h$ fue definida por Glodwasser, Micali y Rivest). Los parámetros $a_0, a_1$ y $n$ se hacen públicos (la función debería admitir un parámetro en el que venga especificado el vector inicial).

In [9]:
h = lambda b, x, a0, a1, n: (am.big_pow(x, 2, n)* (a0**b) * a1**(1 - b)) % n

rand1, rand2 = randint(1, p), randint(1, q)
a0, a1 = am.big_pow(rand1, 2, n), am.big_pow(rand2, 2, n)

print("a0 =", a0, "\na1 =", a1)

a0 = 24017267900686256613169 
a1 = 184247888902510874809


In [10]:
def MerkleDamgard_hash(original_message, vector, a0, a1, n, f):
    
    bin_message, x = str_to_binlist(original_message), vector
    
    for b in bin_message:
        x = f(b, x, a0, a1,n)
        
    return x
    
MerkleDamgard_hash("Lore ipsum dolor sit amet", 987, a0, a1, n, h)

31526974987248429178041

En este ejercicio, cogemos dos elementos aleatorios entre 1 y $p$, y entre 1 y $q$ (donde $n = p\cdot q$) y calculamos el cuadrado de esos números módulo $n$, para obtener dos cuadrados. Tras esto, pasamos a la función de _Merkle-Damgard_ en el que vamos calculando el hash del mensaje gracias a la función $h$. 

Como resultado de la función, devolvemos el último elemento que obtenemos de aplicar la función Hash.

## Ejercicio 5

Sea $p$ el menor primo entero mayor o igual que tu número de identidad, y sea $q$ el primer primo mayor o igual que tu fecha de nacimiento (AAAAMMDD). Selecciona $e$ tal que $gcd(e, (p -1)(q -1)) = 1$. Define la función RSA $$f:\mathbb{Z}_n \rightarrow \mathbb{Z}_n, x \mapsto x^e$$

Calcula el inverso de 1234567890.

In [13]:
p, q =search_prime(int(environ["ID"])), search_prime(int(environ["BIRTH"])) 

φ, e = (p - 1) * (q - 1), randint(1, q)

while am.ext_euclides(e, φ)[0] != 1:
    e = randint(1, q)
    
d = am.inverse(e, φ) + φ

inverse  = am.big_pow(1234567890, d, p*q)
print(inverse)
print(am.big_pow(inverse, e, p*q))

321572152786058
1234567890


En este caso, para el cálculo de la preimagen, partimos de que conocemos el valor de $n$, donde $n = p \cdot q$. Esto se conoce como el _módulo del criptosistema_. Al conocer $p$ y $q$, también conoceremos $\varphi(n) = (p-1)\cdot(q -1)$. Conocido esto, podemos calcular el _exponente de cifrado_, que en este caso es $e$.

Con todos estos datos, podemos calcular $d$ que es el exponente de descifrado. El origen de calcular esto, es que RSA cifra el sistema con la función $f$ definida anteriormente, donde cifra elevando un número al _exponente de cifrado_, mientras que el descifrado se calcula elevando el número cifrado a $d$, donde $d$ tiene una relación matemática con $e$ definida de la siguiente manera $$e\cdot d \equiv 1 \bmod \varphi(n)$$

Si despejamos la congruencia, podemos obtener $d$, y realizando la operación $ 123456790^d\bmod n$, obtendremos su preimagen.

## Ejercicio 6


Sea $n=50000000385000000551$, y sabemos que una inversa de $\mathbb{Z}_n \rightarrow \mathbb{Z}_n, x \mapsto x^5$ es $x \mapsto x^10000000074000000101$ (esto es, conoces tanto la llave pública como la llave privada de la función RSA). Encuentra $p$ y $q$ usando el método explicado en "_Notes on cryptography" Page 92_. Compara este prodecimiento con el algoritmo de Miller-Rabin y el ejercicio 3.

In [68]:
n, e, d = 50000000385000000551, 5, 10000000074000000101

def broke_RSA(n, e, d):
    # Let de - 1 = 2^a * b and take a random x: 0 < x < n 
    # calculate the gcd(x,n)
    a, b = am.bifactor(d*e - 1)
    x = randint(1, n)

    if am.ext_euclides(x, n)[0] != 1:
        # we found a factor of n
        p, q = x, n//x
    else:
        y = am.big_pow(x, b, n)
        z = y
        while y % n != 1:
            z = y
            y = am.big_pow(y, 2, n)
            if y % n == (n - 1):
                raise ValueError("The algorithm has failed")
        
        p, q = am.ext_euclides(z + 1, n)[0], am.ext_euclides(z - 1, n)[0]
    
    return p, q

p, q = broke_RSA(n, e, d)
p*q == n

True